In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
pred_result = pd.read_csv('TSA_decided.csv')
pred_result['id'] = pred_result['Unnamed: 0']+1

pred_result.set_index('id',inplace=True)
pred_result.drop('Unnamed: 0',axis=1, inplace=True)

pred_result
candidates = pd.read_csv('n_sup_decided.csv')
candidates['n_id'] = [int(x[1:]) for x in candidates['id'].values]
selected = candidates.loc[range(50)]
selected
selected.supply_amount.values.sum()/5
avg_need_eqC = (2.82*10**4)*0.72
'每周生产需求(eqC)：'+str((2.82*10**4)*0.72)
trans = pd.read_csv('trans.csv')
def replace0(x):
    if x == 0:
        return np.nan
    else:
        return x
    
notnull = 8*240 - trans.isnull().sum().sum()
avg_los = (trans.sum().values[1:].sum()/notnull)*0.01

'有运输时的平均货损率：'+str(avg_los)

selected_pred = pred_result.loc[selected['n_id'].values]
selected_pred
month_sup_50 = selected_pred.sum(axis=0)
month_sup_50.sum()
need_array_eqC = np.array([avg_need_eqC for i in range(24)])
plt.figure(figsize=[30,4])
plt.xticks(range(1,25))


plt.bar(x=range(1,25), height=month_sup_50, label='month_sup_50', color='darkred')
plt.bar(x=range(1,25), height=need_array_eqC, label='need_array_eqC', color='darkgray',alpha=0.7)

plt.legend()
plt.title('50 Need & Supply Barplot')
plt.savefig('50 Need & Supply Barplot.jpg')
selected_pred.sort_index(inplace=True)
selected_pred.to_csv('for_matlab.csv')
import numpy as np 
from scipy import optimize as op 
selected_pred = selected_pred.T

selected_pred
len(selected_pred)
selected_pred[3].sum()
for i in range(len(selected_pred)):
    if i<=22:
        selected_pred.iloc[i+1] = selected_pred.iloc[i+1] + selected_pred.iloc[i]
selected_pred
np.array(-selected_pred).shape
import cvxpy as cp

c = np.array([1 for i in range(50)])

a = np.array(selected_pred)

b = np.array([avg_need_eqC*i for i in range(1,25)])

x = cp.Variable(50,integer=True)

obj = cp.Minimize(c*x)

cons =[a*x>=b, x>=0, x<=1]

prob =cp.Problem(obj, cons)

prob.solve( solver='GLPK_MI', verbose=True)

print('opt_obj', prob.value)
print('opt_x', x.value)
selected_pred = selected_pred.T
selected_pred['opt_res'] = x.value
selected_pred
opt_res1_df = pd.DataFrame()
opt_res1_df['opt'] = x.value
opt_res1_df['id'] = selected_pred.index
opt_res1_df.to_csv('opt_res1_df.csv')
opt_res1_df
order_eqC = pd.read_csv('supply_eqC.csv')


order_eqC['id'] = [int(x[1:]) for x in order_eqC['供应商ID'].values]
selected_eqC = order_eqC.set_index('id').loc[opt_res1_df.id]
selected_eqC
selected_eqC = selected_eqC['材料分类']
selected_eqC
recat_selected = pd.concat([selected_eqC,pred_result.loc[opt_res1_df.id]], axis=1)
idx = recat_selected.index

recat_selected = recat_selected.set_index('材料分类')
recat_selected.loc['A'] = recat_selected.loc['A']/1.2
recat_selected.loc['B'] = recat_selected.loc['B']/1.091

recat_selected['id'] = idx
recat_selected
# recat_selected.to_csv('recat_selected_pred.csv')
n_selected = recat_selected.copy()
n_selected.loc['A'] = n_selected.loc['A']/0.6
n_selected.loc['B'] = n_selected.loc['B']/0.66
n_selected.loc['C'] = n_selected.loc['C']/0.72
n_selected['id'] = idx
n_selected['mat'] =n_selected.index
n_selected

n_selected['selected2'] = x.value
nn_selected = n_selected.set_index('selected2').loc[1]
nn_selected = nn_selected.set_index('mat')
nn_selected.sort_index(inplace=True)
nn_selected
nn_selected.to_csv('2_plan_data.csv')
a_temp = np.array(nn_selected.drop('id', axis=1)).T
a_temp.shape
len((list(a_temp[0,:]) + [0 for i in range(37*(23-0))]))
a_temp
aa_temp=[[0 for i in range(888)] for i in range(24)]
len(aa_temp[0])

for i in range(24):
    aa_temp[i] = (list(a_temp[i,:]) + [0 for i in range(37*(23-i))])
    aa_temp[i] = ([0 for i in range(37*(23-(23-i)))] + aa_temp[i])
len(aa_temp[0])
aaa_array = np.array(aa_temp)
aaa_array.shape
c_temp = np.array(nn_selected.drop('id', axis=1).loc['A']).flatten()*0.6*1.2
c_temp.shape
c_temp = np.append(c_temp, np.array(nn_selected.drop('id', axis=1).loc['B']).flatten()*0.66*1.1)
c_temp.shape
c_temp = np.append(c_temp, np.array(nn_selected.drop('id', axis=1).loc['C']).flatten()*0.72)
c_temp.shape
b = np.array([avg_need_eqC/0.72*1.01 for i in range(24)])
b
x_df = pd.DataFrame(aaa_array)
x_df.to_csv('a.csv')
x_df = pd.DataFrame(b)
x_df.to_csv('b.csv')
x_df = pd.DataFrame(c_temp)
x_df.to_csv('c.csv')
# c = c_temp

# a = aaa_array

# b = np.array([avg_need_eqC*1.01 for i in range(24)])

# x = cp.Variable(336,integer=True)

# obj = cp.Minimize(c*x)

# cons =[a*x>=b, 1>=x, x>=0]

# prob =cp.Problem(obj, cons)

# prob.solve( solver='GLPK_MI', verbose=True)

# print('opt_obj', prob.value)
# print('opt_x', x.value)
# x_df = pd.DataFrame(c)
# x_df.to_csv('for_matlab.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'TSA_decided.csv'